In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOConfig, DPOTrainer
from datasets import load_dataset

In [27]:
train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")

Generating test split: 100%|██████████| 1000/1000 [00:00<00:00, 37349.10 examples/s]


In [41]:
train_dataset[0]['chosen']

[{'content': 'Use the pygame library to write a version of the classic game Snake, with a unique twist',
  'role': 'user'},
 {'content': "Sure, I'd be happy to help you write a version of the classic game Snake using the pygame library! Here's a basic outline of how we can approach this:\n\n1. First, we'll need to set up the game display and create a game object that we can use to handle the game's state.\n2. Next, we'll create the game's grid, which will be used to represent the game board. We'll need to define the size of the grid and the spaces within it.\n3. After that, we'll create the snake object, which will be used to represent the player's movement. We'll need to define the size of the snake and the speed at which it moves.\n4. We'll also need to create a food object, which will be used to represent the food that the player must collect to score points. We'll need to define the location of the food and the speed at which it moves.\n5. Once we have these objects set up, we can 

In [ ]:
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [38]:
messages = [
    train_dataset[0]['chosen'][0],
]

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=500)
raw_output = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
print(raw_output.replace("<|im_end|>",""))

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Creating a version of the classic game Snake using the pygame library involves several steps. Below is a simplified version of how you might approach this task. This example will focus on creating a basic snake game with a unique twist: the snake can move in a diagonal direction.

First, ensure you have the pygame library installed. You can install it using pip if you haven't already:

```bash
pip install pygame
```

Now, let's write the code for the Snake game:

```python
import pygame
import random

# Initialize pygame
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Snake size
SNAKE_SIZE = 20

# Snake head position
snake_head = [SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2]

# Snake body
snake_body = [[snake_head[0], snake_head[1]]]

# Food position
food = [random.randint(0, SCREEN_WIDTH - SNAKE_SIZE), random.randint(0, SCREEN_HEIGHT - SNAKE_SIZE)]

# Gam

In [48]:
training_args = DPOConfig(output_dir="Qwen2-0.5B-DPO",no_cuda=True)

/home/albatrosfirst/rlhf_finetuning/.venv/lib/python3.10/site-packages/transformers/training_args.py:1604: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [49]:
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=train_dataset)

: 